<a href="https://colab.research.google.com/github/tottenjordan/stock-pred/blob/master/EDA/stockRet_load_fin_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### pip and package

In [2]:
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib
import talib as ta

Selecting previously unselected package libta-lib0.
(Reading database ... 144465 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 266kB 4.2MB/s 
  Created wheel for ta-lib: filename=TA_Lib-0.4.18-cp36-cp36m-linux_x86_64.whl size=1531822 sha256=8c60f3d2b9716576f4f8971408d1d777347efbb3a0cfc31ba62b312c8d964dcb
  Stored in directory: /root/.cache/pip/wheels/11/cc/88/20dc42c28a07efb297dad4303948ef6dd4a3c7257553db8f09
Successfully built ta-lib


In [3]:
!pip install yfinance

  Created wheel for yfinance: filename=yfinance-0.1.54-py2.py3-none-any.whl size=22409 sha256=69240f5f0b780a7efd52d49893314beccf1e91725b85ce6f68ef78529744edc4
  Stored in directory: /root/.cache/pip/wheels/f9/e3/5b/ec24dd2984b12d61e0abf26289746c2436a0e7844f26f2515c
Successfully built yfinance


In [7]:
!pip install mpl_finance

In [8]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None, 'display.max_rows', None)

# finance data
import yfinance as yf

import matplotlib.pyplot as plt
from matplotlib import dates as mdates
import matplotlib.patches as mpatches

import datetime as dt
from mpl_finance import candlestick_ohlc

import warnings
warnings.filterwarnings("ignore")

### Load Data

In [24]:
# load yahoo data
def load_fin_data(ticker, start_date, end_date, lookup_step=1, progress=False, feature_columns=['Adj Close', 'Volume', 'Open', 'High', 'Low'], 
                  AROON_n=25, BETA_n=5, CCI_n=20, COPPOCK=[11,14], CLV_n=[10,42], CMF_n=20, DPO_n=21, ADX_n=14, RSI_n=14, VORTEX_n=14,
                  STOCHASTIC_n=14,STOCHASTIC_dn=3,DONCHAIN_n=20):
    """
    Loads data from Yahoo Finance source, creates additional feature columns, and creates multiple target columns.
    Params:
        ticker (str/pd.DataFrame): the ticker you want to load, examples include AAPL, TESL, etc.
        lookup_step (int): the future lookup step to predict, default is 1 (e.g next day)
        start_date / end_date (date): first and last dates of the series to download
        progress (bool): shows progress bar for downloading from yahoo, default is False
        feature_columns (list): the list of features to use to feed into the model, default is everything grabbed from yahoo_fin
    """
    # see if ticker is already a loaded stock from yahoo finance
    if isinstance(ticker, str):
        # load it from yahoo_fin library
        df = yf.download(ticker, start=start_date, end=end_date, progress=progress)
    elif isinstance(ticker, pd.DataFrame):
        # already loaded, use it directly
        df = ticker
    else:
        raise TypeError("ticker can be either a str or a `pd.DataFrame` instances")

    # make sure that the passed feature_columns exist in the dataframe
    for col in feature_columns:
        assert col in df.columns, f"'{col}' does not exist in the dataframe."

    # add features e.g., returns and log returns
    df = df.rename(columns={'Adj Close':'Adj_Close'})
    df['returns'] = df['Adj_Close'].pct_change().dropna()
    df['log_returns'] = np.log(df['Adj_Close']).diff()
    df['StatVol_100'] = df['log_returns'].rolling(window=100).std() * np.sqrt(100)
    df['close_3day_mean'] = df['Adj_Close'].rolling(window=3).mean()
    df['close_5day_mean'] = df['Adj_Close'].rolling(window=5).mean()
    df['close_20day_mean'] = df['Adj_Close'].rolling(window=20).mean()
    df['VolumeAvg_3d'] = df['Volume'].rolling(window=3).mean()
    df['VolumeAvg_5d'] = df['Volume'].rolling(window=5).mean()
    df['VolumeAvg_20d'] = df['Volume'].rolling(window=20).mean()
    df['VolumeAvg_100d'] = df['Volume'] / df['Volume'].rolling(window=100).mean()
    df['day_of_week'] = df.index.dayofweek
    df['month'] = df.index.month
    df['returns_signal'] = np.where(df['log_returns'] > 0, 1, 0)

    # Aroon Oscillator
    n = AROON_n ################
    high_max = lambda xs: np.argmax(xs[::-1])
    low_min = lambda xs: np.argmin(xs[::-1])
    df['Days_since_last_High'] = df['High'].rolling(center=False,min_periods=0,window=AROON_n).apply(func=high_max).astype(int)
    df['Days_since_last_Low'] = df['Low'].rolling(center=False,min_periods=0,window=AROON_n).apply(func=low_min).astype(int)
    df['Aroon_Up'] = ((AROON_n-df['Days_since_last_High'])/AROON_n) * 100
    df['Aroon_Down'] = ((AROON_n-df['Days_since_last_Low'])/AROON_n) * 100
    df['Aroon_Oscillator'] = df['Aroon_Up'] - df['Aroon_Down']
    df['AO_Positive'] = df['Aroon_Oscillator'] > 0

    # BoP
    df['BoP'] = (df['Adj_Close'] - df['Open']) / (df['High'] - df['Low'])
    df['VolumePositive'] = df['Open'] < df['Adj_Close']

    # Bolling Bands
    df['BB_Upper'], df['BB_Middle'], df['BB_Lower'] = ta.BBANDS(df['Adj_Close'])
    df['BI'] = (df['Adj_Close'] - df['BB_Lower'])/(df['BB_Upper'] - df['BB_Lower'])

    # CCI
    df['TP'] = (df['High'] + df['Low'] + df['Adj_Close'])/3
    df['SMA_TP'] = df['TP'].rolling(CCI_n).mean()
    df['SMA_STD'] = df['TP'].rolling(CCI_n).std()
    df['CCI'] = (df['TP'] - df['SMA_TP']) / (0.015*df['SMA_STD'])

    # Coppock Curve
    for nd in COPPOCK:
      df['ROC_'+str(nd)] = ((df['Adj_Close'] - df['Adj_Close'].shift(nd))/df['Adj_Close'].shift(nd)) * 100
    df['COPP'] = (df['ROC_14']+df['ROC_11']).ewm(ignore_na=False,span=10,min_periods=10,adjust=True).mean()

    # CLV
    for nd in CLV_n:
      df['CLV_'+str(nd)] = (((df['Adj_Close'] - df['Low'].rolling(window=nd).min()) - \
                             (df['High'].rolling(window=nd).max() - df['Adj_Close'])) / (df['High'].rolling(window=nd).max() \
                                                                                        - df['Low'].rolling(window=nd).min()))
      df['AccVolume_'+str(nd)] = df['Volume'].rolling(nd).sum()
      df['MFV_'+str(nd)] = df['AccVolume_'+str(nd)] * df['CLV_'+str(nd)]
      df['ADL_'+str(nd)] = df['MFV_'+str(nd)].cumsum()

    # CMF
    df['MF_Multiplier'] = (2*df['Adj_Close'] - df['Low'] - df['High'])/(df['High']-df['Low'])
    df['MF_Volume'] = df['MF_Multiplier']*df['Volume']
    df['CMF'] = df['MF_Volume'].rolling(CMF_n).sum()/df['Volume'].rolling(CMF_n).sum()
    df['ADL'] = df['MF_Volume'].cumsum()
    df['ADL_3_EMA'] = df['ADL'].ewm(ignore_na=False,span=3,min_periods=2,adjust=True).mean()
    df['ADL_10_EMA'] = df['ADL'].ewm(ignore_na=False,span=10,min_periods=9,adjust=True).mean()
    df['Chaikin_Oscillator'] = df['ADL_3_EMA'] - df['ADL_10_EMA']
    df['CMF_Positive'] = df['CMF'] > 0

    # ADX - DX
    df['ADX'] = ta.ADX(df['High'], df['Low'],df['Adj_Close'], timeperiod=ADX_n)
    df['Plus_DI'] = ta.PLUS_DI(df['High'], df['Low'],df['Adj_Close'], timeperiod=ADX_n)
    df['Minus_DI'] = ta.MINUS_DI(df['High'], df['Low'],df['Adj_Close'], timeperiod=ADX_n)
    df['DX'] = ta.DX(df['High'], df['Low'],df['Adj_Close'], timeperiod=ADX_n)

    # RSI
    df['RSI'] = ta.RSI(df['Adj_Close'], timeperiod=RSI_n)

    # MACD
    df['macd'], df['macdsignal'], df['macdhist'] = ta.MACD(df['Adj_Close'], fastperiod=12, slowperiod=26, signalperiod=9)

    # OBV
    df['OBV'] = ta.OBV(df['Adj_Close'], df['Volume'])/10**6

    # Vortex
    df['Prior_Low'] = df['Low'].shift()
    df['Prior_High'] = df['High'].shift()
    df['+VM'] = abs(df['High'] - df['Prior_Low'])
    df['-VM'] = abs(df['Low'] - df['Prior_High'])
    df['+VM_'+str(VORTEX_n)] = df['+VM'].rolling(VORTEX_n).sum()
    df['-VM_'+str(VORTEX_n)] = df['-VM'].rolling(VORTEX_n).sum()
    df['HL'] = df['High'] - df['Low']
    df['HC'] = abs(df['High'] - df['Adj_Close'].shift())
    df['LC'] = abs(df['Low'] -df['Adj_Close'].shift())
    df['TR'] = df[['HL','HC','LC']].max(axis=1)
    df['TR_'+str(VORTEX_n)] = df['TR'].rolling(VORTEX_n).sum()
    df['+VI_'+str(VORTEX_n)] = df['+VM_'+str(VORTEX_n)]/df['TR_'+str(VORTEX_n)]
    df['-VI_'+str(VORTEX_n)] = df['-VM_'+str(VORTEX_n)]/df['TR_'+str(VORTEX_n)]

    # Stochastic Oscillator
    smin = df['Low'].rolling(STOCHASTIC_n).min()
    smax = df['High'].rolling(STOCHASTIC_n).max()
    df['stoch_k'] = 100 * (df['Adj_Close'] - smin) / (smax - smin)
    df['stoch_d'] = df['stoch_k'].rolling(STOCHASTIC_dn).mean()

    # Donchain Channel
    DONCHAIN_n
    df['Upper_Channel_Line'] = pd.Series.rolling(df['High'], window=DONCHAIN_n).max()
    df['Lower_Channel_Line'] = pd.Series.rolling(df['Low'], window=DONCHAIN_n).min()
    df['Middle_Channel_Line'] = (df['Upper_Channel_Line'] + df['Lower_Channel_Line'])/2

    # Kelten Channel
    df['EMA'] = ta.EMA(df['Adj_Close'], timeperiod=20)
    df['ATR'] = ta.ATR(df['High'], df['Low'], df['Adj_Close'], timeperiod=10)
    df['Upper_Line'] = df['EMA'] + 2*df['ATR']
    df['Lower_Line'] = df['EMA'] - 2*df['ATR']

    #### Market Indices ####
    # BETA

    # GSPC returns
    gspc = '^GSPC'
    mk_gspc = yf.download(gspc, start=start_date, end=end_date, progress=progress)
    mk_gspc = mk_gspc.rename(columns={'Adj Close':'Adj_Close'})
    mk_gspc['returns'] = mk_gspc['Adj_Close'].pct_change().dropna()
    covar_gspc = df['returns'].rolling(BETA_n).cov(mk_gspc['returns'])
    variance_gspc = mk_gspc['returns'].rolling(BETA_n).var()
    df['BetaGSPC'] = covar_gspc / variance_gspc

    # DJI returns
    dow = '^DJI'
    mk_dow = yf.download(dow,start,end, progress=progress)
    mk_dow = mk_dow.rename(columns={'Adj Close':'Adj_Close'})
    mk_dow['returns'] = mk_dow['Adj_Close'].pct_change().dropna()
    covar_dow = df['returns'].rolling(BETA_n).cov(mk_dow['returns'])
    variance_dow = mk_dow['returns'].rolling(BETA_n).var()
    df['BetaDJI'] = covar_dow / variance_dow

    # VIX
    vix = '^VIX'
    mk_vix = yf.download(vix,start,end, progress=progress)
    mk_vix = mk_vix.rename(columns={'Adj Close':'Adj_Close'})
    mk_vix = mk_vix.drop(columns=['Volume'])
    mk_vix['returns'] = mk_vix['Adj_Close'].pct_change().dropna()
    covar_vix = df['returns'].rolling(BETA_n).cov(mk_vix['returns'])
    variance_vix = mk_vix['returns'].rolling(BETA_n).var()
    df['BetaVIX'] = covar_vix / variance_vix

    # IXIC returns
    ixic = '^IXIC'
    mk_ixic = yf.download(ixic,start,end, progress=progress)
    mk_ixic = mk_ixic.rename(columns={'Adj Close':'Adj_Close'})
    mk_ixic['returns'] = mk_ixic['Adj_Close'].pct_change().dropna() 
    covar_ixic = df['returns'].rolling(BETA_n).cov(mk_ixic['returns'])
    variance_ixic = mk_ixic['returns'].rolling(BETA_n).var()
    df['BetaIXIC'] = covar_ixic / variance_ixic 

    # DPO
    df['DPO_'+str(ticker)] = df['Adj_Close'].shift(int((0.5 * DPO_n) + 1)) - df['Adj_Close'].rolling(DPO_n).mean()
    df['DPO_GSPC'] = mk_gspc['Adj_Close'].shift(int((0.5 * DPO_n) + 1)) - mk_gspc['Adj_Close'].rolling(DPO_n).mean()
    df['DPO_DOW'] = mk_dow['Adj_Close'].shift(int((0.5 * DPO_n) + 1)) - mk_dow['Adj_Close'].rolling(DPO_n).mean()
    df['DPO_VIX'] = mk_vix['Adj_Close'].shift(int((0.5 * DPO_n) + 1)) - mk_vix['Adj_Close'].rolling(DPO_n).mean()
    df['DPO_IXIC'] = mk_ixic['Adj_Close'].shift(int((0.5 * DPO_n) + 1)) - mk_ixic['Adj_Close'].rolling(DPO_n).mean()

    # add target columns (labels), shifting by `lookup_step`
    df['Future1_Close'] = df['Adj_Close'].shift(-lookup_step)
    df['Future1_Returns'] = df['returns'].shift(-lookup_step)
    df['Future1_LogReturns'] = df['log_returns'].shift(-lookup_step)
    df['Future1_ReturnSignal'] = df['returns_signal'].shift(-lookup_step)
    
    # drop NaNs
    df.dropna(inplace=True)

    df = df.drop(['Days_since_last_High', 'Days_since_last_Low','TP', 'SMA_TP', 'SMA_STD',
                  'MF_Multiplier','MF_Volume','ADL','ADL_3_EMA','ADL_10_EMA','Prior_Low',
                  'Prior_High','+VM','-VM','+VM_14','-VM_14','TR','TR_14','HL','HC','LC','ATR'],axis=1)

    # convert to integer
    df['Future1_ReturnSignal'] = df['Future1_ReturnSignal'].astype(int)

    # Prefix df cols
    df.columns = [str(ticker) + '_' + str(col) for col in df.columns]
    mk_gspc.columns = ['GSPC_' + str(col) for col in mk_gspc.columns]
    mk_dow.columns = ['DJI_' + str(col) for col in mk_dow.columns]
    mk_vix.columns = ['VIX_' + str(col) for col in mk_vix.columns]
    mk_ixic.columns = ['IXIC_' + str(col) for col in mk_ixic.columns]

    # join Ticker + Market Indices dfs
    df = df.join(mk_gspc).join(mk_dow).join(mk_vix).join(mk_ixic)

    return df

In [25]:
# input
ticker = 'CGNX'
start = '2017-04-30' # consistent start: 2018-02-17... start well before to get long term trends
end = '2020-07-13'

# load data
cgnx_df = load_fin_data(ticker, start_date=start, end_date=end, lookup_step=1, progress=False, feature_columns=['Adj Close', 'Volume', 'Open', 'High', 'Low'])

print("Shape:", cgnx_df.shape)
cgnx_df.head()

Shape: (704, 103)


,CGNX_Open,CGNX_High,CGNX_Low,CGNX_Close,CGNX_Adj_Close,CGNX_Volume,CGNX_returns,CGNX_log_returns,CGNX_StatVol_100,CGNX_close_3day_mean,CGNX_close_5day_mean,CGNX_close_20day_mean,CGNX_VolumeAvg_3d,CGNX_VolumeAvg_5d,CGNX_VolumeAvg_20d,CGNX_VolumeAvg_100d,CGNX_day_of_week,CGNX_month,CGNX_returns_signal,CGNX_Aroon_Up,CGNX_Aroon_Down,CGNX_Aroon_Oscillator,CGNX_AO_Positive,CGNX_BoP,CGNX_VolumePositive,CGNX_BB_Upper,CGNX_BB_Middle,CGNX_BB_Lower,CGNX_BI,CGNX_CCI,CGNX_ROC_11,CGNX_ROC_14,CGNX_COPP,CGNX_CLV_10,CGNX_AccVolume_10,CGNX_MFV_10,CGNX_ADL_10,CGNX_CLV_42,CGNX_AccVolume_42,CGNX_MFV_42,CGNX_ADL_42,CGNX_CMF,CGNX_Chaikin_Oscillator,CGNX_CMF_Positive,CGNX_ADX,CGNX_Plus_DI,CGNX_Minus_DI,CGNX_DX,CGNX_RSI,CGNX_macd,CGNX_macdsignal,CGNX_macdhist,CGNX_OBV,CGNX_+VI_14,CGNX_-VI_14,CGNX_stoch_k,CGNX_stoch_d,CGNX_Upper_Channel_Line,CGNX_Lower_Channel_Line,CGNX_Middle_Channel_Line,CGNX_EMA,CGNX_Upper_Line,CGNX_Lower_Line,CGNX_BetaGSPC,CGNX_BetaDJI,CGNX_BetaVIX,CGNX_BetaIXIC,CGNX_DPO_CGNX,CGNX_DPO_GSPC,CGNX_DPO_DOW,CGNX_DPO_VIX,CGNX_DPO_IXIC,CGNX_Future1_Close,CGNX_Future1_Returns,CGNX_Future1_LogReturns,CGNX_Future1_ReturnSignal,GSPC_Open,GSPC_High,GSPC_Low,GSPC_Close,GSPC_Adj_Close,GSPC_Volume,GSPC_returns,DJI_Open,DJI_High,DJI_Low,DJI_Close,DJI_Adj_Close,DJI_Volume,DJI_returns,VIX_Open,VIX_High,VIX_Low,VIX_Close,VIX_Adj_Close,VIX_returns,IXIC_Open,IXIC_High,IXIC_Low,IXIC_Close,IXIC_Adj_Close,IXIC_Volume,IXIC_returns
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-09-21,56.615002,57.035000,56.375000,56.700001,56.083492,1208200,0.001678,0.001677,0.193698,56.160975,56.089427,54.008799,1.146800e+06,1223360.0,1251030.0,0.748678,3,9,1,92.0,12.0,80.0,True,-0.805317,False,56.935124,56.089427,55.243730,0.496491,82.833083,5.803319,4.065343,11.911367,0.249289,12140600.0,3.026516e+06,-8.059658e+07,0.713371,69585600.0,4.964036e+07,8.584470e+08,-1.087271,-5.520180e+06,False,37.956316,27.500079,10.873547,43.328019,69.576634,1.607199,1.545446,0.061753,34.4244,0.898410,0.640158,67.750603,69.320672,57.674999,50.705002,54.190001,54.123573,56.776161,51.470985,1.425875,1.572587,0.291312,0.971706,-0.822387,-9.244757,-208.299293,0.656191,3.857654,56.098328,0.000265,0.000264,1,2507.159912,2507.159912,2499.000000,2500.600098,2500.600098,2930860000,-0.003046,22414.019531,22419.509766,22356.550781,22359.230469,22359.230469,287390000,-0.002381,9.74,10.21,9.54,9.67,9.67,-0.011247,6448.569824,6448.569824,6405.299805,6422.689941,6422.689941,1772790000,-0.005166
2017-09-22,56.529999,57.044998,56.505001,56.715000,56.098328,541200,0.000265,0.000264,0.187175,56.057116,56.235818,54.261273,9.719333e+05,1040200.0,1226190.0,0.345139,4,9,1,88.0,8.0,80.0,True,-0.799395,False,56.695494,56.235818,55.776141,0.350449,79.132505,5.173850,4.707831,11.542333,0.020700,11935400.0,2.470637e+05,-8.034951e+07,0.716043,68589400.0,4.911295e+07,9.075599e+08,-1.148982,-5.597516e+06,False,38.345927,26.143714,10.316163,43.410873,69.641355,1.569957,1.550348,0.019608,34.9656,0.872723,0.643427,68.051218,67.216133,57.674999,51.345001,54.510000,54.311645,56.891275,51.732014,1.266696,1.642522,0.344776,0.884851,-0.769635,-12.455171,-256.755859,0.702857,1.345819,54.451431,-0.029357,-0.029797,0,2497.260010,2503.469971,2496.540039,2502.219971,2502.219971,2865960000,0.000648,22334.070312,22364.310547,22299.580078,22349.589844,22349.589844,297140000,-0.000431,9.90,10.20,9.50,9.59,9.59,-0.008273,6401.439941,6429.540039,6400.810059,6426.919922,6426.919922,1639100000,0.000659
2017-09-25,56.689999,56.900002,54.840000,55.049999,54.451431,1089200,-0.029357,-0.029797,0.188973,55.544417,55.806537,54.442778,9.462000e+05,1014160.0,1184130.0,0.700914,0,9,0,84.0,4.0,80.0,True,-1.086682,False,57.190934,55.806537,54.422139,0.010579,27.016375,2.029472,2.494869,10.266335,-1.274123,11538600.0,-1.470159e+07,-9.505111e+07,0.419438,68414200.0,2.869554e+07,9.362555e+08,-1.058551,-5.597594e+06,False,36.579842,23.378664,17.773447,13.620728,55.521030,1.391510,1.518581,-0